In [ ]:
def evaluate_PDF(rv, x=4):
    '''Input: a random variable object, standard deviation
       output : x and y values for the normal distribution
       '''
    
    # Identify the mean and standard deviation of random variable 
    mean = rv.mean()
    std = rv.std()

    # Use numpy to calculate evenly spaced numbers over the specified interval (4 sd) and generate 100 samples.
    xs = np.linspace(mean - x*std, mean + x*std, 100)
    
    # Calculate the peak of normal distribution i.e. probability density. 
    ys = rv.pdf(xs)

    return xs, ys # Return calculated values

# Cohen's  d
Cohen’s D is one of the most common ways to measure effect size. As an effect size, Cohen's d is typically used to represent the magnitude of differences between two (or more) groups on a given variable, with larger values representing a greater differentiation between the two groups on that variable.

The basic formula to calculate Cohen’s  dd  is:

dd  = effect size (difference of means) / pooled standard deviation

The denominator is the standardiser, and it is important to select the most appropriate one for a given dataset. The pooled standard deviation is the average spread of all data points around their group mean (not the overall mean).


effect_size = mean_difference / sd



# Interpreting  d
Most people don't have a good sense of how big  d=2.0d=2.0  is. If you are having trouble visualizing what the result of Cohen’s D means, use these general “rule of thumb” guidelines (which Cohen said should be used cautiously):

Small effect = 0.2

Medium Effect = 0.5

Large Effect = 0.8

Here is an excellent online visualization tool developed by Kristoffer Magnusson to help interpret the results of cohen's  dd statistic.

In [ ]:
def Cohen_d(group1, group2):

    # Compute Cohen's d.

    # group1: Series or NumPy array
    # group2: Series or NumPy array

    # returns a floating point number 

    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()

    # Calculate the pooled threshold as shown earlier
    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    
    # Calculate Cohen's d statistic
    d = diff / np.sqrt(pooled_var)
    
    return d


def plot_pdfs(cohen_d=2):
    """Plot PDFs for distributions that differ by some number of stds.
    
    cohen_d: number of standard deviations between the means
    """
    group1 = scipy.stats.norm(0, 1)
    group2 = scipy.stats.norm(cohen_d, 1)
    xs, ys = evaluate_PDF(group1)
    plt.fill_between(xs, ys, label='Group1', color='#ff2289', alpha=0.7)

    xs, ys = evaluate_PDF(group2)
    plt.fill_between(xs, ys, label='Group2', color='#376cb0', alpha=0.7)
    
    o, s = overlap_superiority(group1, group2)
    print('overlap', o)
    print('superiority', s)
    
    
from scipy import stats
import numpy as np
import seaborn as sns

def one_sample_ttest(sample, popmean, alpha):

    # Visualize sample distribution for normality 
    sns.set(color_codes=True)
    sns.set(rc={'figure.figsize':(12,10)})
    sns.distplot(sample)
    
    # Population mean 
    mu = popmean
    
    # Sample mean (x̄) using NumPy mean()
    x_bar= sample.mean()

    # Sample Standard Deviation (sigma) using Numpy
    sigma = np.std(sample)
    
    # Degrees of freedom
    df = len(sample) - 1
    
    #Calculate the critical t-value
    t_crit = stats.t.ppf(1 - alpha, df=df)
    
    #Calculate the t-value and p-value
    results = stats.ttest_1samp(a= sample, popmean= mu)         
    
    if (results[0]>t_crit) and (results[1]<alpha):
        print ("Null hypothesis rejected. Results are statistically significant with t-value =", 
                round(results[0], 2), "critical t-value =", t_crit, "and p-value =", np.round((results[1]), 10))
    else:
        print ("Null hypothesis is True with t-value =", 
                round(results[0], 2), ", critical t-value =", t_crit, "and p-value =", np.round((results[1]), 10))
    

# Two Sampled T-Test

In [ ]:
#It is always a good idea to draw the probability distributions for samples to visually inspect the differences 
#present between mean and standard deviation. Plot both samples' distributions and inspect the overlap using seaborn 
#to get an idea how different the samples might be from one another.

sns.set(color_codes=True)
sns.set(rc={'figure.figsize':(12,10)})
sns.distplot(control) # Blue distribution
sns.distplot(experimental) # Green distribution


"""As a reminder the five steps to performing a hypothesis test are:

Set up null and alternative hypotheses
Choose a significance level
Calculate the test statistic
Determine the critical or p-value (find the rejection region)
Compare t-value with critical t-value to reject or fail to reject the null hypothesis"""

"""Now, create some functions to calculate the t-statistic. The first function to create is one 
that calculates the variance for a single sample."""

def sample_variance(sample):
    sample_mean = np.mean(sample)
    return np.sum((sample - sample_mean) **2)/ (len(sample) -1)
    
#Using sample_variance, you can now write another function pooled_variance to calculate $S_{p}^{2}$

def pooled_variance(sample1, sample2):
    n_1, n_2 = len(sample1), len(sample2)
    var_1, var_2 = sample_variance(sample1), sample_variance(sample2)
    return ((n_1-1) * var_1 + (n_2-1)* var_2)/((n_1 + n_2)-2)

def twosample_tstatistic(expr, ctrl):
    exp_mean, ctrl_mean = np.mean(expr), np.mean(ctrl)
    pool_var = pooled_variance(expr, ctrl)
    n_e, n_c = len(expr), len(ctrl)
    num = exp_mean - ctrl_mean
    denom = np.sqrt(pool_var * ((1/n_e)+(1/n_c)))
    return num / denom

#Write a function visualize_t that uses matplotlib to display a standard t-distribution with vertical lines identifying each critical value that signifies the rejection region.

# Visualize p_value

def visualize_t(t_stat, n_control, n_experimental):
    
    """
    Visualize the critical t values on a t distribution
    
    Parameters
    -----------
    t-stat: float
    n_control: int
    n_experiment: int
    
    Returns
    ----------
    None
    
    """

    # initialize a matplotlib "figure"
    fig = plt.figure(figsize=(8,5))
    ax = fig.gca()
    # generate points on the x axis between -4 and 4:
    xs = np.linspace(-4, 4, 500)

    # use stats.t.pdf to get values on the probability density function for the t-distribution
    
    ys= stats.t.pdf(xs, (n_control+n_experimental-2), 0, 1)
    ax.plot(xs, ys, linewidth=3, color='darkred')

    ax.axvline(t_stat, color='black', linestyle='--', lw=5)
    ax.axvline(-t_stat, color='black', linestyle='--', lw=5)

    plt.show()
    return None

#n_control = len(control)
#n_experimental = len(experimental)
#visualize_t(t_stat, n_control, n_experimental)

"""Given a t-value and a degrees of freedom, you can use the "survival function" sf of scipy.stats.t 
(aka the complementary CDF) to compute the one-sided p-value. For the two-sided p-value, 
just double the one-sided p-value.
"""
## Calculate p_value
# Lower tail comulative density function returns area under the lower tail curve
lower_tail = stats.t.cdf(-1.89, (50+50-2), 0, 1)
# Upper tail comulative density function returns area under upper tail curve
upper_tail = 1. - stats.t.cdf(1.89, (50+50-2), 0, 1)

p_value = lower_tail+upper_tail
print(p_value)




# TYPE 1 Error and Type 2 Error

In [ ]:
def type_1_error(population, num_tests, alpha_set):
    """
    Parameters
    ----------
    population: ndarray
        A random normal distribution
    num_tests: int
        The number of hypothesis tests to be computed
    alpha_set: list
        List of alpha levels
    
    Returns
    ----------
    sig_tests : DataFrame
        A dataframe containing the columns 'type_1_error', 'p_value', and 'alpha'
    """
    columns = ['type_1_error','p_value','alpha']
    sig_tests = pd.DataFrame(columns=columns)
    counter = 0
    
    for i in range(1,num_tests+1):
        
        for alpha in alpha_set:
            
            # take two samples from the same population
            samp1 = np.random.choice(population,100,replace=True)
            samp2 = np.random.choice(population,100,replace=True)
            
            # test sample means
            result = stats.ttest_ind(samp1, samp2)
            
            # evaluate whether null hypothesis is rejected or not
            if result[1] < alpha:
                 sig_tests.loc[counter] = [1, result[1], alpha]
            else:
                 sig_tests.loc[counter] = [0, result[1], alpha]

            counter += 1
            
    return sig_tests


def type_2_error(population, population_2, num_tests, alpha_set):
    
    """
    Parameters
    ----------
    population: ndarray
        A random normal distribution
    population_2: ndarray
        A different random normal distribution
    num_tests: int
        The number of hypothesis tests to be computed
    alpha_set: list
        List of alpha levels
    
    Returns
    ----------
    sig_tests : DataFrame
        A dataframe containing the columns 'type_2_error', 'p_value', and 'alpha'
    """
    
    columns = ['type_2_error','p_val','alpha']
    sig_tests = pd.DataFrame(columns=columns)
    counter = 0
    
    for i in range(1,num_tests+1):
        
        for alpha in alpha_set:
            
            # take two samples from the same population
            samp1 = np.random.choice(population,100,replace=True)
            samp2 = np.random.choice(population_2,100,replace=True)
            
            # test sample means
            result = stats.ttest_ind(samp1, samp2)
            
            # evaluate whether null hypothesis is rejected or not
            if result[1] > alpha:
                 sig_tests.loc[counter] = [1, result[1], alpha]
            else:
                 sig_tests.loc[counter] = [0, result[1], alpha]

            counter += 1
            
    return sig_tests

If you decide to use a large value for alpha :

Increases the chance of rejecting the null hypothesis
The risk of a Type II error (false negative) is REDUCED
Risk of a Type I error (false positive) is INCREASED
Similarly, if you decide to use a very small value of alpha, it'll change the outcome as:

Increases the chance of accepting the null hypothesis
The risk of a Type I error (false positive) is REDUCED
Risk of a Type II error (false negative) is INCREASED
From above, you can see that in statistical hypothesis testing, the more you try and avoid a Type I error (false positive), the more likely a Type II error (false negative) will occur.

# Key Takeaways
Some of the key takeaways from this section include:

It's important to have a sound approach to experimental design to be able to determine the significance of your findings
Start by examining any existing research to see if it can shed light on the problem you're studying
Start with a clear alternative and null hypothesis for your experiment to "prove"
It's important to have a thoughtfully selected control group from the same population for your trial to distinguish effect from variations based on population, time or other factors
Sample size needs to be selected carefully to ensure your results have a good chance of being statistically significant
Your results should be reproducible by other people and using different samples from the population
The p-value for an outcome determines how likely it is that the outcome could be due to chance
The alpha value is the marginal threshold at which we're comfortable rejecting the null hypothesis
An alpha of 0.05 is a common choice for many experiments
Effect size measures just the size in difference between two groups under observation, whereas statistical significance combines effect size with sample size
A one sample t-test is used to determine whether a sample comes from a population with a specific mean.
A two-sample t-test is used to determine if two population means are equal
Type 1 errors (false positives) are when we accept an alternative hypothesis which is actually false
The alpha that we pick is the likelihood that we will get a type 1 error due to random chance
Type 2 errors (false negatives) are when we reject an alternative hypothesis which is actually true

# Statistical Power
The power of a statistical test is defined as the probability of rejecting the null hypothesis, given that it is indeed false.

 Statsmodels has some convenient build in methods for calculating the power of a t-test and plotting power curves.

In [ ]:
from statsmodels.stats.power import TTestIndPower, TTestPower
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid') #Nice background styling on plots

power_analysis = TTestIndPower()

power_analysis.plot_power(dep_var="nobs",
                          nobs = np.array(range(5,1500)),
                          effect_size=np.array([.05, .1, .2,.3,.4,.5]),
                          alpha=0.05)
plt.show()



In addition to plotting a full curve, you can also calculate specific values. Simply don't specify one of the four parameters.

In [ ]:
#Calculate power
power_analysis.solve_power(effect_size=.2, nobs1=80, alpha=.05)

#Calculate sample size required
power_analysis.solve_power(effect_size=.2, alpha=.05, power=.8)

#Calculate minimum effect size to satisfy desired alpha and power as well as respect sample size limitations
power_analysis.solve_power(nobs1=25, alpha=.05, power=.8)

#Calculate alpha (less traditional)
power_analysis.solve_power(nobs1=25, effect_size=.3, power=.8)



# Welches T-Test

Recall that z-tests are also appropriate for statistics, such as the mean, which can be assumed to be normally distributed. However, when sample sizes are low (n_observations<30), the t-test is more appropriate, as it has heavier tails. Even with this modification, remember that there are still several assumptions to the model. Most notably, traditional t-tests assume that sample sizes and sample variances between the two groups are equal. When these assumptions are not met, Welch's T-test is generally a more reliable test.

the student's  tt -test assumes that the samples are of equal size and equal variance ( so , mean as well ). When these assumptions are not met, then Welch's  tt -test provides a more accurate p-value.



In [ ]:
def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """

    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)


def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. """
    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator


import numpy as np
import scipy.stats as stats



def p_value_welch_ttest(a, b, two_sided=False):
    """Calculates the p-value for Welch's t-test given two samples.
    By default, the returned p-value is for a one-sided t-test. 
    Set the two-sided parameter to True if you wish to perform a two-sided t-test instead.
    """
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p
    
    


# ANOVA 

Loading the Data
As usual, we start by loading in a dataset of our sample observations. This particular table is of salaries in IT and has 4 columns:

S - the individuals salary

X - years of experience

E - education level (1-Bachelors, 2-Masters, 3-PHD)

M - management (0-no management, 1-yes management)


In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

formula = 'S ~ C(E) + C(M) + X'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

NameError: name 'df' is not defined

### Reading the Table¶
For now , simply focus on the outermost columns. On the left, you can see our various groups, and on the right, the probability that the factor is indeed influential. Values < .05 (or whatever we set  αα  to) indicate rejection of the null hypothesis. 

### A 2-Category ANOVA F-Test is Equivalent to a 2-Tailed t-Test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-Test! So, the p-value in the table should be identical to your calculation above.

Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations.


In [ ]:

#Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
#Compare the p-value to that of the t-test above. 
#They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

## Generating Multiple T-Tests
While the 2-category ANOVA test is identical to a 2-tailed t-Test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features:


In [ ]:

for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)
('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


"""While bad practice, examine the effects of calculating multiple t-tests with the various
combinations of these. To do this, generate all combinations of the above groups. 
For each pairwise combination, calculate the p-value of a 2 sided t-test. 
Print the group combinations and their associated p-value for the two-sided t-test.
"""
#Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
#for all combinations of the supplement-dose groups listed above. 
#(Since there isn't a control group, compare each group to every other group.)

from itertools import combinations

groups = [group[0] for group in df.groupby(['supp', 'dose'])['len']]
combos = combinations(groups, 2)
for combo in combos:
    supp1 = combo[0][0]
    dose1 = combo[0][1]
    supp2 = combo[1][0]
    dose2 = combo[1][1]
    sample1 = df[(df.supp == supp1) & (df.dose == dose1)]['len']
    sample2 = df[(df.supp == supp2) & (df.dose == dose2)]['len']
    p = stats.ttest_ind(sample1, sample2, equal_var=False)[1]
    print(combo, p)


# Key Takeaways
Remember that the section began where the last left off, examining the relationship between  αα , power, effect size and sample size. As you saw, these 4 quantities form a deterministic relationship; know any 3, and you can caulculate the fourth. While a lower alpha value will lead to fewer type I errors, and a higher power will lead to fewer type II errors, in practice these are often set to common default standards due to exploding sample sizes required to detect various effect sizes. Some common thresholds used are:

Setting alpha equal to .05 (or .01)
Requiring power values of .8 or greater
After a thorough investigation of this relationship, you then also saw an alternative t-test, Welch's t-test which can be used for comparing samples of different sizes of with different variances. While the formula was a bit complicated, the most important piece to remember is that when the assumptions that sample size and sample variance are equal for the two samples is violated, use Welch's t-test rather then student's t-test.

Aside from ensuring that the assumptions of a t-test are met, it's also important to know how type I errors are compounded if you perform multiple A/B tests. This is known as the multiple comparison problem and you saw that type I errors compound under multiple tests. So while the probability of a type I error is equal to  αα  for any one test, the collective probability that there is at least 1 type I error continues to increase as you perform more tests, further detracting from the confidence that you have uncovered a meaningful relationship. In order to account for this, you can use stricter criteria when defining  αα  such as the Bonferroni Correction. Alternatively, ANOVA is equivalent to a 2 sided t-test when comparing 2 groups, but also generalizes appropriately to multiple group comparisons.

Summary
Remember that simply observing a low p-value is not meaningful in and of itself. There are a number of factors to take into consideration when interpreting the results of a statistical test, from alpha, power, sample size, effect size, and the formulation of the problem itself. Good hypothesis testing requires careful thought and design.

## Note that parametric means numeric , non-parametric is non-numeric may be ranks or some sort of value
